In [18]:
import pandas as pd
import numpy as np

In [103]:
df_CellBlind_pre = pd.read_csv('../../data/drug_sensitivity_lung_CellBlind_test&prediction.csv', index_col=0)
df_drug_mean_std = pd.read_csv('../../data/gdsc_drug_mean_std.csv')
# 将df_MixedSet_pre和df_drug_mean_std合并，以df_MixedSet_pre的drug，df_drug_mean_std的Drug_Name为键
df_CellBlind_pre = pd.merge(df_CellBlind_pre, df_drug_mean_std, left_on='drug', right_on='Drug_Name')
# 去掉Drug_Name列
df_CellBlind_pre = df_CellBlind_pre.drop(columns=['Drug_Name'])
# 分别计算IC50和prediction的Z-score
df_CellBlind_pre['IC50_Z'] = (df_CellBlind_pre['IC50'] - df_CellBlind_pre['IC50_mean']) / df_CellBlind_pre['IC50_std']
df_CellBlind_pre['prediction_Z'] = (df_CellBlind_pre['prediction'] - df_CellBlind_pre['IC50_mean']) / df_CellBlind_pre['IC50_std']
# 去掉IC50_mean, IC50_std列
df_CellBlind_pre = df_CellBlind_pre.drop(columns=['IC50_mean', 'IC50_std'])
# 按照prediction从小到大排序
df_CellBlind_pre = df_CellBlind_pre.sort_values(by='prediction')
df_CellBlind_pre.head()

,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
1309,Docetaxel,SIGM5,-5.402247,AMOL,-6.514387,-0.778892,-1.215265
2265,Docetaxel,GA10,-5.996559,BL,-6.412953,-1.012083,-1.175465
1000,SN-38,KE37,-6.589709,ATLL,-6.360167,-1.228809,-1.164997
10600,SN-38,LU135,-4.976839,SCLC,-6.263311,-0.780435,-1.138071
7876,Daporinad,NALM6,-7.519153,BLL,-6.246553,-1.491129,-1.037505


In [96]:
# 筛选出OncotreeCode = UCEC PAAD HCC GB PCM BLCA
df_CellBlind_pre = df_CellBlind_pre[df_CellBlind_pre['OncotreeCode'].isin(['UCEC', 'PAAD', 'HCC', 'GB', 'PCM', 'AMOL'])]
# 去除细胞系为EN的数据
# df_CellBlind_pre = df_CellBlind_pre[df_CellBlind_pre['cell_line'].str.contains('EN') == False]
print(df_CellBlind_pre.shape)
df_CellBlind_pre.head()

(3214, 7)


,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
1309,Docetaxel,SIGM5,-5.402247,AMOL,-6.514387,-0.778892,-1.215265
57,Daporinad,OPM2,-5.123417,PCM,-6.204997,-0.637159,-1.022693
6748,Romidepsin,EJM,-4.803048,PCM,-6.138752,0.435656,-0.895059
2583,Romidepsin,NOMO1,-4.850046,AMOL,-6.121387,0.388833,-0.877759
1403,SN-38,SIGM5,-3.500200,AMOL,-6.113964,-0.369934,-1.096553


In [104]:
df_CellBlind_pre[['cell_line','OncotreeCode']].value_counts()

cell_line  OncotreeCode
PC14       LUAD            233
LS1034     COAD            219
NCIH1299   LCLC            218
LCLC103H   LCLC            218
HMVII      VMM             218
                          ... 
SIGM5      AMOL            176
SKNSH      NBL             153
RPMI8402   TLL             152
OC314      SOC             151
KYSE150    ESCC            151
Name: count, Length: 61, dtype: int64

In [97]:
# 去除细胞系为EN、COLO684、MFE296、PATU8902、HUH7  LN229  MFE296

df_CellBlind_pre = df_CellBlind_pre[~df_CellBlind_pre['cell_line'].isin(['EN', 'PATU8902', 'HUH7','LN229','MFE296'])]
df_CellBlind_pre.shape

(2128, 7)

In [86]:
cell_lines = df_CellBlind_pre['cell_line'].unique()
print(len(cell_lines))
print(cell_lines)

11
['OPM2' 'EJM' 'NOMO1' 'SKHEP1' 'COLO684' 'LN229' 'SNU398' 'HUPT3' 'MFE296'
 'A172' 'HUPT4']


In [87]:
# 找出所有细胞系drug的交集
cell_lines = df_CellBlind_pre['cell_line'].unique()
drug_intersection = df_CellBlind_pre[df_CellBlind_pre['cell_line'] == cell_lines[0]]['drug'].unique()
for cell_line in cell_lines[1:]:
    drug_intersection = np.intersect1d(drug_intersection, df_CellBlind_pre[df_CellBlind_pre['cell_line'] == cell_line]['drug'].unique())
    
print(len(drug_intersection))
print(drug_intersection)

212
['5-Fluorouracil' '5-azacytidine' 'A-366' 'AGI-5198' 'AGI-6780' 'AGK2'
 'AMG-319' 'AT13148' 'AZ6102' 'AZ960' 'AZD1208' 'AZD1332' 'AZD2014'
 'AZD3759' 'AZD4547' 'AZD5153' 'AZD5363' 'AZD5438' 'AZD5582' 'AZD5991'
 'AZD6738' 'AZD7762' 'AZD8055' 'AZD8186' 'Acetalax' 'Afatinib'
 'Afuresertib' 'Alisertib' 'Alpelisib' 'Avagacestat' 'Axitinib' 'BI-2536'
 'BIBR-1532' 'BMS-345541' 'BMS-536924' 'BMS-754807' 'Bleomycin'
 'Bleomycin (50 uM)' 'Bortezomib' 'Bosutinib' 'Bromosporine' 'Buparlisib'
 'CCT007093' 'CPI-637' 'CZC24832' 'Camptothecin' 'Carmustine' 'Cediranib'
 'Cisplatin' 'Crizotinib' 'Cyclophosphamide' 'Cytarabine' 'Dabrafenib'
 'Dacarbazine' 'Dactinomycin' 'Dactolisib' 'Dasatinib' 'Dihydrorotenone'
 'Dinaciclib' 'Docetaxel' 'Doramapimod' 'EHT-1864' 'EPZ004777' 'EPZ5676'
 'Elephantin' 'Entinostat' 'Entospletinib' 'Epirubicin' 'Erlotinib'
 'Fludarabine' 'Foretinib' 'Fulvestrant' 'GDC0810' 'GSK-LSD1'
 'GSK1904529A' 'GSK2110183B' 'GSK2578215A' 'GSK2606414' 'GSK2801'
 'GSK2830371' 'GSK343' '

In [88]:
df_CellBlind_pre[['OncotreeCode']].value_counts()

OncotreeCode
GB              434
HCC             434
PAAD            434
UCEC            434
PCM             433
AMOL            217
Name: count, dtype: int64

# 选取药物 三种：

In [98]:
df_Ibrutinib = df_CellBlind_pre[df_CellBlind_pre['drug'] == 'Vorinostat']
df_Ibrutinib.head(20)

,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
1430,Vorinostat,SIGM5,-0.791738,AMOL,0.340588,-2.090015,-1.051702
204,Vorinostat,OPM2,0.487461,PCM,0.431825,-0.917023,-0.968040
6791,Vorinostat,EJM,1.039647,PCM,0.708544,-0.410683,-0.714296
2627,Vorinostat,NOMO1,1.327706,AMOL,0.722464,-0.146541,-0.701532
7159,Vorinostat,COLO684,1.865851,UCEC,0.918729,0.346924,-0.521562
3248,Vorinostat,SNU398,2.260098,HCC,1.358779,0.708438,-0.118048
3201,Vorinostat,SKHEP1,1.492565,HCC,1.661037,0.004631,0.159115
1644,Vorinostat,HUPT3,1.834141,PAAD,1.680360,0.317847,0.176834
4844,Vorinostat,A172,1.133355,GB,1.969847,-0.324755,0.442286
3060,Vorinostat,HUPT4,3.022006,PAAD,1.984361,1.407088,0.455595


In [99]:
df_Dactolisib = df_CellBlind_pre[df_CellBlind_pre['drug'] == 'Dactolisib']
df_Dactolisib.head(20)

,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
1305,Dactolisib,SIGM5,-2.131960,AMOL,-3.364974,-0.434496,-1.296872
56,Dactolisib,OPM2,-3.593117,PCM,-2.902245,-1.456437,-0.973237
6642,Dactolisib,EJM,-1.655538,PCM,-2.851671,-0.101284,-0.937866
2478,Dactolisib,NOMO1,-1.726593,AMOL,-2.540188,-0.150980,-0.720013
3098,Dactolisib,SKHEP1,-2.457305,HCC,-2.461159,-0.662044,-0.664739
3108,Dactolisib,SNU398,-1.803821,HCC,-2.407605,-0.204994,-0.627283
1495,Dactolisib,HUPT3,0.641964,PAAD,-2.306914,1.505602,-0.556860
4783,Dactolisib,A172,-2.988897,GB,-2.083570,-1.033842,-0.400652
7010,Dactolisib,COLO684,-1.912310,UCEC,-1.609293,-0.280871,-0.068940
2911,Dactolisib,HUPT4,-0.010738,PAAD,-0.573503,1.049098,0.655497


In [100]:
df_AZD8055 = df_CellBlind_pre[df_CellBlind_pre['drug'] == 'AZD8055']
df_AZD8055.head(20)

,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
2445,AZD8055,NOMO1,0.430937,AMOL,-1.267641,0.339510,-0.898880
23,AZD8055,OPM2,-2.233541,PCM,-1.101641,-1.603093,-0.777854
6609,AZD8055,EJM,-0.351157,PCM,-0.853110,-0.230695,-0.596656
3113,AZD8055,SNU398,-1.390495,HCC,-0.311307,-0.988450,-0.201642
1462,AZD8055,HUPT3,1.938377,PAAD,-0.296429,1.438546,-0.190794
4799,AZD8055,A172,-0.938582,GB,-0.271662,-0.658972,-0.172737
3129,AZD8055,SKHEP1,-0.479479,HCC,-0.270897,-0.324251,-0.172180
2878,AZD8055,HUPT4,1.880757,PAAD,0.177216,1.396536,0.154528
6977,AZD8055,COLO684,-1.173098,UCEC,0.476101,-0.829951,0.372437


In [101]:
df_PD0325901 = df_CellBlind_pre[df_CellBlind_pre['drug'] == 'PD0325901']
df_PD0325901.head(20)

,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
1376,PD0325901,SIGM5,1.735574,AMOL,-3.630404,0.182151,-2.319071
2564,PD0325901,NOMO1,-3.638064,AMOL,-1.522511,-2.322642,-1.336527
1580,PD0325901,HUPT3,0.394802,PAAD,0.227930,-0.442818,-0.520601
2996,PD0325901,HUPT4,1.542918,PAAD,0.263844,0.092349,-0.503860
3223,PD0325901,SKHEP1,1.883252,HCC,0.497211,0.250988,-0.395082
3208,PD0325901,SNU398,1.656975,HCC,1.659574,0.145514,0.146726
140,PD0325901,OPM2,1.179499,PCM,2.074874,-0.077050,0.340308
6727,PD0325901,EJM,2.709521,PCM,2.181923,0.636134,0.390206
4849,PD0325901,A172,1.347392,GB,2.521830,0.001210,0.548646
7095,PD0325901,COLO684,3.834328,UCEC,3.013659,1.160435,0.777900


In [102]:
df_Paclitaxel = df_CellBlind_pre[df_CellBlind_pre['drug'] == 'Paclitaxel']
df_Paclitaxel.head(20)

,drug,cell_line,IC50,OncotreeCode,prediction,IC50_Z,prediction_Z
1384,Paclitaxel,SIGM5,-5.174181,AMOL,-5.935402,-1.149637,-1.527094
148,Paclitaxel,OPM2,-4.238882,PCM,-5.033156,-0.685862,-1.079709
2572,Paclitaxel,NOMO1,-4.682721,AMOL,-4.908457,-0.905943,-1.017876
3091,Paclitaxel,SNU398,-3.520420,HCC,-4.592042,-0.329608,-0.860979
6735,Paclitaxel,EJM,-4.191512,PCM,-4.114496,-0.662373,-0.624184
3089,Paclitaxel,SKHEP1,-3.548045,HCC,-3.857410,-0.343306,-0.496707
7103,Paclitaxel,COLO684,-2.347088,UCEC,-2.876524,0.252197,-0.010327
3004,Paclitaxel,HUPT4,-1.193314,PAAD,-2.489099,0.824304,0.181780
1588,Paclitaxel,HUPT3,-2.431350,PAAD,-2.480799,0.210415,0.185896
4773,Paclitaxel,A172,-4.343105,GB,-1.362199,-0.737542,0.740562
